Необходимые программы: Anaconda Prompt
Откройте Anaconda Prompt и выполните следующию команду (команду также можно выполнить из ячейки Jupyter)
pip install psycopg2
(для MacOS):
pip install psycopg2-binary

In [21]:
# Создаем соединение c базой данных -
import psycopg2 # драйвер для postgresql
import getpass  # прячем параль, чтобы не писать его в параметрах
import pandas as pd
import datetime

#Обратите внимание на параметр database=... имя базы данных должно совпадать с названием Вашей БД. У меня она по умолчанию назвалась
# postgres, но в кодах с семинаров мы использовали test
conn=psycopg2.connect(user='postgres',
                      database='postgres',
                      host='localhost',
                      port='5432',
                      password=getpass.getpass('Insert password: ')
                        )
print(conn.get_dsn_parameters())

Insert password: ········
{'user': 'postgres', 'dbname': 'postgres', 'host': 'localhost', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'}


In [44]:
# Достаем данные из БД
# Замечание касательно отбора максимального купона - в таблице Quotes по некоторым облигациям есть данные по различным купонным
# выплатам, при этом в bond_ description эти облигации имеют тип купона - постоянный.
# Также мы сразу фильтруем вытягиваемые данные, отсеивая те, которые имеют статус "Погашен", что бы потом не повторять это в функции

query = """ SELECT *
            FROM (SELECT MAX("CPN") AS "CPN", "ISIN", max("BUYBACKDATE") AS "BUYBACKDATE",
                  max("BUYBACKPRICE") AS "BUYBACKPRICE"
                  FROM public.quotes_task  
                  GROUP BY "ISIN") AS cpn
            INNER JOIN(SELECT "ISINCode", "FaceValue", "HaveOffer", "AmortisedMty", 
                       "CouponPerYear", "NumCoupons", "EndMtyDate", "CouponType", "IssuerName"
                       FROM public.bond_description_task 
                       WHERE "Status"!='Погашен') AS bonds
            ON cpn."ISIN" = bonds."ISINCode" """

# Загружаем в python

data = pd.read_sql_query(query, conn)
data = pd.DataFrame(data)
data.head(5)

,CPN,ISIN,BUYBACKDATE,BUYBACKPRICE,ISINCode,FaceValue,HaveOffer,AmortisedMty,CouponPerYear,NumCoupons,EndMtyDate,CouponType,IssuerName
0,NaN,RU000A0E5690,None,0.0,RU000A0E5690,1000.0,True,False,4,16,2009-06-09,Переменный,Нутринвестхолдинг
1,41.1400,RU000A0GN9B5,2013-02-21,100.0,RU000A0GN9B5,1000.0,True,False,2,20,2016-02-18,Переменный,Мосэнерго (ТГК-3)
2,30.2918,RU000A0JNGU3,2016-01-06,100.0,RU000A0JNGU3,1000.0,True,False,4,40,2016-07-06,Переменный,Банк ВТБ
3,NaN,RU000A0JNN80,None,0.0,RU000A0JNN80,1000.0,False,False,2,6,2009-09-17,Постоянный,МГ Групп
4,40.1100,RU000A0JNYN1,None,NaN,RU000A0JNYN1,1000.0,False,False,2,31,2022-06-11,Фиксированный,Москва Правительство


In [45]:
# Поле ISINCode было нужно нам для джойна, сейчас его можно выбросить
data = data.drop(['ISINCode'], axis=1)
data.head(5)

,CPN,ISIN,BUYBACKDATE,BUYBACKPRICE,FaceValue,HaveOffer,AmortisedMty,CouponPerYear,NumCoupons,EndMtyDate,CouponType,IssuerName
0,NaN,RU000A0E5690,None,0.0,1000.0,True,False,4,16,2009-06-09,Переменный,Нутринвестхолдинг
1,41.1400,RU000A0GN9B5,2013-02-21,100.0,1000.0,True,False,2,20,2016-02-18,Переменный,Мосэнерго (ТГК-3)
2,30.2918,RU000A0JNGU3,2016-01-06,100.0,1000.0,True,False,4,40,2016-07-06,Переменный,Банк ВТБ
3,NaN,RU000A0JNN80,None,0.0,1000.0,False,False,2,6,2009-09-17,Постоянный,МГ Групп
4,40.1100,RU000A0JNYN1,None,NaN,1000.0,False,False,2,31,2022-06-11,Фиксированный,Москва Правительство


In [46]:
# Объявляем процедуру, которая будет переводить дату, введенную пользователем, из строкового формата в дату

def get_date(date_entry):
    year, month, day = map(int, date_entry.split('-'))
    return datetime.date(year, month, day)

In [47]:
# Для начала общий концепт: по каждой облигации у нас есть информация по ее Maturity date, а также периодичность выплаты купонов
# и их общее число. Тогда все даты платежей можно задать путем отнимания от EndMtyDate числа дней, соответствующих периодичности
# выплаты купона. И так повторить N раз - число NumCoupons. Это будет полная матрица платежей по одной облигации. Однако нас
# интересует ее усеченная версия, где точкой отсечения является введенная пользователем дата

def my_fun(issuer_name, terminal_date):
    # составим список дат платежей для каждой облигации
    
    # получим нужные облигации
    terminal_date = get_date(terminal_date)
    obligations = data[data['IssuerName'] == issuer_name]
    
    
    matrix = dict()
    
    # для каждой облигации формируем список дат платежей
    for index, obligation in obligations.iterrows():
        matrix[obligation['ISIN']] = []
        
        # сначала определяем интервалы, через которые будут отсекаться даты относительно последнего платежа
        period = 365 // obligation['CouponPerYear']
        
        # определяем дату последнего платежа (с учетом условия про наличие оферты)
        payment_date = None
        if obligation['BUYBACKDATE'] is not None:
            payment_date = obligation['BUYBACKDATE']
        else:
            payment_date = obligation['EndMtyDate']
        
        # Основной цикл, который, вроде бы, реализует "концепт", изложенный выше - "нарезка" дат
        for i in range(obligation['NumCoupons']):
            
            # даты больше введеной пропускаем
            if payment_date > terminal_date:
                pass
            else:
                matrix[obligation['ISIN']].append(payment_date)
            
            # считаем следующую дату выплаты
            payment_date = payment_date - datetime.timedelta(days=period)
    return matrix    

In [48]:
# Теперь необходимо сформировать подобие сводной таблицы, где строками будут ИСИНы облигаций, а столбцами - даты платежей
def build_pivot(matrix_dict):
    
    # сначала получим список всех дат всех облигаций и упорядочим по возрастанию
    date_set=set()
    for isin in matrix_dict:
        date_set.update(matrix_dict[isin])
    date_set = list(date_set)
    date_set.sort()
    
    # создаем сводную таблицу
    rs = pd.Index(matrix_dict.keys(), name="rows")
    cs = pd.Index(date_set, name="columns")
    df = pd.DataFrame(index = rs, columns = cs)
    
    # заполняем дата фрейм
    for isin in matrix_dict:
        
        # для каждой облигации берем список дат
        datelist = matrix_dict[isin]
        
        # и заполняем его верным значением в зависимости от даты
        obligation = data[data['ISIN'] == isin].iloc[0]
        cpn = obligation['CPN']
        face_value = obligation['FaceValue']
        
        # определяем верную дату
        terminal_date = None
        if obligation['BUYBACKDATE'] is not None:
            terminal_date = obligation['BUYBACKDATE']
        else:
            terminal_date = obligation['EndMtyDate']
            
        for the_date in datelist:
        # и наконец заполняем значение выплаты
            if the_date == terminal_date:
                df.at[isin, the_date] = cpn + face_value
            else:
                df.at[isin, the_date] = cpn
    return df

In [50]:
x=my_fun('Банк ВТБ', '2015-06-06')
y=build_pivot(x)
y

columns,2006-04-07,2006-04-19,2006-07-07,2006-07-19,2006-10-06,2006-10-18,2006-10-20,2007-01-05,2007-01-17,2007-01-19,...,2015-01-07,2015-01-09,2015-01-20,2015-02-17,2015-03-27,2015-03-31,2015-04-08,2015-04-10,2015-04-21,2015-05-19
rows,,,,,,,,,,,,,,,,,,,,,
RU000A0JNGU3,NaN,30.2918,NaN,30.2918,NaN,30.2918,NaN,NaN,30.2918,NaN,...,30.2918,NaN,NaN,NaN,NaN,NaN,30.2918,NaN,NaN,NaN
RU000A0JTK20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,20.32,NaN,NaN,NaN,NaN,NaN,20.32,NaN
RU000A0JTN01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,19.7,NaN,NaN,NaN,NaN,NaN,19.7
RU000A0JU773,NaN,NaN,NaN,NaN,NaN,NaN,35.28,NaN,NaN,35.28,...,NaN,35.28,NaN,NaN,NaN,NaN,NaN,35.28,NaN,NaN
RU000A0JUQE1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,23.81,NaN,NaN,NaN,NaN
RU000A0JV3Q3,42.3836,NaN,42.3836,NaN,42.3836,NaN,NaN,42.3836,NaN,NaN,...,NaN,NaN,NaN,NaN,42.3836,NaN,NaN,NaN,NaN,NaN
